# BRIEF
IT NOW CAN PERFORM REGRESSION AND multi class CLASSIFICATION



In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Reshape and add channel dimension
train_images = train_images.reshape((60000, 28, 28, 1))  # [B, H, W, C]
test_images = test_images.reshape((10000, 28, 28, 1))    # [B, H, W, C]

# Convert to [B, C, H, W] format  # [B, C, H, W]

# Normalize pixel values to the range [0, 1]
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

train_images = train_images.transpose(0, 3, 1, 2)
test_images = test_images.transpose(0, 3, 1, 2)

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Print shapes
print("Train images shape:", train_images.shape)  # Should be (60000, 1, 28, 28)
print("Train labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)    # Should be (10000, 1, 28, 28)
print("Test labels shape:", test_labels.shape)

In [ ]:
print(type(train_labels))

In [ ]:
import numpy as np
from core.Models import Model
from core.nn import Linear, Conv2d, MaxPool2d , batchnorm2d, Softmax, Relu,Flatten
from core.optim import sgd, adam
from core.loss import get_loss_fn

class ResNetWithResiduals(Model):
    def __init__(self):
        super().__init__()

        # Initial Convolution
        self.conv1 = Conv2d(input_channels=1, output_channels=8, kernel_size=3, stride=1, padding=1, initialize_type='xavier',bias=False)
        self.bn1 = batchnorm2d(8)
        self.relu1 = Relu()

        # Residual Block 1
        self.conv2 = Conv2d(input_channels=8, output_channels=16, kernel_size=3, stride=1, padding=1, initialize_type='xavier')
        self.bn2 = batchnorm2d(16)
        self.relu2 = Relu()
        self.res1 = Conv2d(input_channels=8, output_channels=16, kernel_size=1, stride=1, padding=0, initialize_type='xavier')  # Adjust channels
        self.max1 = MaxPool2d(kernel_size=2, stride=2)

        # Residual Block 2
        self.conv3 = Conv2d(input_channels=16, output_channels=32, kernel_size=3, stride=1, padding=1, initialize_type='xavier')
        self.bn3 = batchnorm2d(32)
        self.relu3 = Relu()

        self.conv4 = Conv2d(input_channels=32, output_channels=64, kernel_size=3, stride=1, padding=1, initialize_type='xavier')
        self.bn4 = batchnorm2d(64)
        self.relu4 = Relu()
        self.res2 = Conv2d(input_channels=16, output_channels=64, kernel_size=1, stride=1, padding=0, initialize_type='xavier')  # Adjust channels
        self.max2 = MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected Layer
        self.flatten = Flatten()
        self.linear1 = Linear(64 * 7 * 7, 100, initialize_type='xavier', activation='relu')
        self.linear2 = Linear(100, 10, initialize_type='xavier', activation='softmax')


    def forward(self, x):
        # Initial conv
        x1 = self.conv1(x)
        x1 = self.bn1(x1)
        x1 = self.relu1(x1)

        # Residual Block 1
        x2 = self.conv2(x1)
        x2 = self.bn2(x2)
        x2 = self.relu2(x2)
        res1 = self.res1(x1)  # Match channel dimensions
        x2 = x2 + res1  # Add residual connection
        x2 = self.max1(x2)

        # Residual Block 2
        x3 = self.conv3(x2)
        x3 = self.bn3(x3)
        x3 = self.relu3(x3)

        x4 = self.conv4(x3)
        x4 = self.bn4(x4)
        x4 = self.relu4(x4)
        res2 = self.res2(x2)  # Match channel dimensions
        x4 = x4 + res2  # Add residual connection
        x4 = self.max2(x4)

        # Fully Connected Layer
        x4 = self.flatten(x4)
        x4 = self.linear1(x4)
        x4 = self.linear2(x4)


        return x4

# Instantiate model
model_resnet_residuals = ResNetWithResiduals()


In [ ]:
from core.Datasets import Dataset
train_dataset = Dataset(train_images, train_labels, batch_size=32, shuffle=True)

In [ ]:
optimizer = adam(model_resnet_residuals.parameters(), learning_rate=0.001)
loss_fn = get_loss_fn('categorical_bce')
num_epochs = 1
def train():
    for epoch in range(num_epochs):
        train_dataset.reset()  # Reset dataset iterator and reshuffle if needed
        epoch_loss = 0.0
        num_batches = 0
        for X_batch, y_batch in train_dataset:
            optimizer.zero_grad()
            out = model_resnet_residuals(X_batch)
            loss_tensor = loss_fn.sparse_categorical_cross_entropy(y_batch, out)
            epoch_loss += loss_tensor.data
            num_batches += 1
            loss_tensor.backward()
            optimizer.step()
        avg_loss = epoch_loss / num_batches
        
        print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {avg_loss:.4f}")




In [ ]:
train()

In [ ]:
print(model_resnet_residuals.layers.keys())

In [ ]:
from core.config import Config

In [ ]:
def test():
    model_resnet_residuals.test()
    print(Config.TEST)
    x = model_resnet_residuals(test_images)
    y_pred = x.data
    y_pred_classes = np.argmax(y_pred, axis=1, keepdims=True)
    y_true_classes = np.argmax(test_labels, axis=1, keepdims=True)  # Assuming test_labels are integers
    accuracy = np.mean(y_pred_classes == y_true_classes)
    print(f"Accuracy on test set: {accuracy:.4f}")

test()

In [ ]:

model_resnet_residuals.view_graph(input_data=test_images[:1],filename="model_graph", format="png", view=True)

In [ ]:
model_resnet_residuals.save_model(filepath="model_resnet.h5")

# Augmentation Module


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 (only training data for demonstration)
(x_train, y_train), _ = cifar10.load_data()

# Select a sample image (CIFAR-10 images are 32x32 RGB)
sample_img = x_train[0]

# Import Compose and all Preprocessing transforms from your package.
from Preprocessing.transforms import (
    Compose, GaussianBlur, GaussianNoise, Normalize, 
    RandomAffine, RandomCrop, RandomErasing, RandomHorizontalFlip, 
    RandomRotation, RandomVerticalFlip, Resize, ToTensor
)

# Create a dictionary mapping transformation names to their instances.
# For demonstration, we force application by setting probabilities to 1.0.
aug_transforms = {
    "RandomHorizontalFlip": RandomHorizontalFlip(p=1.0),
    "RandomVerticalFlip":   RandomVerticalFlip(p=1.0),
    "RandomCrop":           RandomCrop((28, 28)),  # Crop to 28x28 (from 32x32)
    "RandomRotation":       RandomRotation(degrees=30),
    "RandomAffine":         RandomAffine(degrees=20, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10),
    "GaussianBlur":         GaussianBlur(sigma=(0.5, 1.5)),
    "GaussianNoise":        GaussianNoise(std=(0.01, 0.05)),
    "RandomErasing":        RandomErasing(p=1.0, scale=(0.02, 0.15), ratio=(0.3, 3.3), value=0)
}

# Prepare the plot: one row per Preprocessing (each row shows Original and Augmented)
n_transforms = len(aug_transforms)
fig, axes = plt.subplots(nrows=n_transforms, ncols=2, figsize=(8, n_transforms * 3))

for idx, (name, transform) in enumerate(aug_transforms.items()):
    # Create a Compose pipeline with a single transform.
    pipeline = Compose([transform])
    
    # Left column: display the original image.
    axes[idx, 0].imshow(sample_img)
    axes[idx, 0].set_title("Original")
    axes[idx, 0].axis("off")
    
    # Right column: apply the Compose pipeline and display the result.
    aug_img = pipeline(sample_img)
    if hasattr(aug_img, "data"):
        aug_img = aug_img.data
    axes[idx, 1].imshow(np.clip(aug_img, 0, 255).astype(np.uint8))
    axes[idx, 1].set_title(name)
    axes[idx, 1].axis("off")

plt.tight_layout()
plt.show()


# testing

In [ ]:
import torch
from torchvision.models import resnet18, ResNet18_Weights

# Download the pre-trained ResNet-18 model
pytorch_resnet18 = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1, progress=True)
state_dict = pytorch_resnet18.state_dict()

In [ ]:
for key,_ in state_dict.items():
    print(key,"size:",state_dict[key].size())

#remove num_batches_tracked from state_dict


In [ ]:
# Load the state_dict (assuming you already have it)
filtered_state_dict = {k: v for k, v in state_dict.items() if "num_batches_tracked" not in k}
print(len(filtered_state_dict))
# Print the new state_dict keys to verify
for key in filtered_state_dict.keys():
    print("size:", filtered_state_dict[key].size())

# Save or use the filtered state_dict as needed


In [ ]:
#https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/

from pretrained.resnet18 import resnet18
from core.Models import Model
model = resnet18(pretrained=True)
model.test()

In [ ]:
from PIL import Image
import numpy as np
from core.tensor import Tensor
label = 153
imagepath = r"C:\Users\ahmed\Downloads\n02085936_Maltese_dog (1).JPEG"

img = Image.open(imagepath)
img = img.resize((224, 224))
img = np.array(img)
img = np.expand_dims(img, axis=0)
img = np.transpose(img, (0, 3, 1, 2))  # convert to (batch_size, channels, height, width)
img = img / 255.0
img = Tensor(img)

output = model(img)
y_pred = output.data
print(y_pred.shape)
#print top 5 predictions
print("Top 5 predictions:", np.argsort(y_pred[0])[::-1][:5])
# pred_label = np.argmax(y_pred[0], axis=0, keepdims=False)
# print("Predicted label:", pred_label)


In [ ]:
from pretrained.vgg16 import VGG16
vgg = VGG16(pretrained=True)

In [ ]:
from PIL import Image
import numpy as np
from core.tensor import Tensor
label = 153
imagepath = r"C:\Users\ahmed\Downloads\n02085936_Maltese_dog (1).JPEG"

img = Image.open(imagepath)
img = img.resize((224, 224))
img = np.array(img)
img = np.expand_dims(img, axis=0)
img = np.transpose(img, (0, 3, 1, 2))  # convert to (batch_size, channels, height, width)
img = img / 255.0
img = Tensor(img)

output = vgg(img)
y_pred = output.data
pred_label = np.argmax(y_pred[0], axis=0, keepdims=False)
print("Top 5 predictions:", np.argsort(y_pred[0])[::-1][:5])


In [3]:
from core.Models import Model
from core.nn import PositionalEmbedding , PatchEmbedding , LayerNorm

class model(Model):
    def __init__(self):
        super().__init__()
        self.patch_embedding = PatchEmbedding()
        self.positional_embedding = PositionalEmbedding(self.patch_embedding.n_patches, 768)
        self.layer_norm = LayerNorm(768)

    def forward(self, x):
        x = self.patch_embedding(x)
        x = self.positional_embedding(x)
        x = self.layer_norm(x)
        return x

In [4]:
from core.tensor import Tensor
import numpy as np
x = Tensor(np.random.rand(1, 3, 224, 224))
model = model()
output = model(x)
output.backward()

AttributeError: 'Tensor' object has no attribute 'std'

In [ ]:
import numpy as np
from core.tensor import Tensor
from core.nn import MultiHeadAttention
attention_layer = MultiHeadAttention(dmodel=8, nheads=2, masked=True)

# Generate a random input tensor
B, T, dmodel = 2, 7, 8  # Batch size = 4, Sequence length = 5, Embedding size = 8
x = np.random.rand(B, T, dmodel)  # Random input
x = Tensor(x)
# Forward pass
output = attention_layer(x)

# Print results
print("Input shape: ", x.shape)        # Expected: (4, 5, 8)
print("Output shape:", output.shape)   # Expected: (4, 5, 8)
# print("output type:", output)

# print("Output values:\n", output.data) 

In [ ]:
grad = output.backward()
print(output.grad.shape)  # Expected: (4, 5, 8)
print("outputGrad values:\n", output.grad)
print(x.grad.shape)  # Expected: (4, 5, 8)
print("xGrad values:\n", x.grad)